In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz
import fastbook
fastbook.setup_book()

In [2]:
#hide
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [3]:
#| export
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
creds = ''

In [4]:
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)
     

In [5]:
comp = 'bluebook-for-bulldozers'
path = URLs.path(comp)
path

Path('/home/petewin/.fastai/archive/bluebook-for-bulldozers')

In [6]:
#hide
Path.BASE_PATH = path

In [7]:
from kaggle import api

if not path.exists():
    path.mkdir(parents=true)
    api.competition_download_cli(comp, path=path)
    shutil.unpack_archive(str(path/f'{comp}.zip'), str(path))

path.ls(file_type='text')

(#7) [Path('TrainAndValid.csv'),Path('Valid.csv'),Path('random_forest_benchmark_test.csv'),Path('Test.csv'),Path('median_benchmark.csv'),Path('ValidSolution.csv'),Path('Machine_Appendix.csv')]

In [13]:
for i in path.ls(file_type='text'):
    print(i)

/home/petewin/.fastai/archive/bluebook-for-bulldozers/TrainAndValid.csv
/home/petewin/.fastai/archive/bluebook-for-bulldozers/Valid.csv
/home/petewin/.fastai/archive/bluebook-for-bulldozers/random_forest_benchmark_test.csv
/home/petewin/.fastai/archive/bluebook-for-bulldozers/Test.csv
/home/petewin/.fastai/archive/bluebook-for-bulldozers/median_benchmark.csv
/home/petewin/.fastai/archive/bluebook-for-bulldozers/ValidSolution.csv
/home/petewin/.fastai/archive/bluebook-for-bulldozers/Machine_Appendix.csv


In [14]:
path.ls()

(#15) [Path('TrainAndValid.csv'),Path('Train.7z'),Path('Valid.csv'),Path('Data Dictionary.xlsx'),Path('Train.zip'),Path('bluebook-for-bulldozers.zip'),Path('random_forest_benchmark_test.csv'),Path('TrainAndValid.7z'),Path('Test.csv'),Path('Valid.7z')...]

In [16]:
df = pd.read_csv(path/'TrainAndValid.csv', low_memory=False)

- SalesID: the uniue identifier of the sale
- MachineID: the unique identifier of a machine.  A machine can be sold multiple times
- saleprice: what the machine sold for at auction (only provided in train.csv)
- saledate: the date of the sale

In [17]:
df.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [18]:
df['ProductSize'].unique()

array([nan, 'Medium', 'Small', 'Large / Medium', 'Mini', 'Large', 'Compact'], dtype=object)

In [19]:
sizes = 'Large', 'Large / Medium', 'Medium', 'Small', 'Mini', 'Compact'

In [20]:
df['ProductSize'] = df['ProductSize'].astype('category')

In [23]:
df['ProductSize'].cat.set_categories(sizes, ordered=True, inplace=True)

/tmp/ipykernel_17413/3701344575.py:1: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.


Kaggle tells us what metric to use: root mean squared log error (RMSLE) between the actual and predicted auction prices